In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

시작 전 test_quality_data.csv 파일의 문자열 형태는 엑셀 수식변환을 사용하여 시작하였습니다.

## Test quality data -1 제외 평균

In [28]:
test_qul  = pd.read_csv('LG/test_quality_data.csv')
test_qul

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,2.020110e+13,30000,04.33.1261,0.0,0.0,0.0,0,0,0.0,0,5,0,0,2,0,0
1,2.020110e+13,30000,04.33.1261,0.0,0.0,0.0,0,0,0.0,0,5,0,0,2,0,0
2,2.020110e+13,30000,04.33.1261,0.0,0.0,0.0,0,0,0.0,0,5,0,0,2,0,0
3,2.020110e+13,30000,04.33.1261,0.0,0.0,0.0,0,0,0.0,0,5,0,0,2,0,0
4,2.020110e+13,30000,04.33.1261,0.0,0.0,0.0,0,0,0.0,0,5,0,0,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747967,2.020110e+13,44997,04.22.1750,0.0,0.0,0.0,0,0,2.0,0,0,0,0,3,0,0
747968,2.020110e+13,44997,04.22.1750,0.0,0.0,0.0,0,0,0.0,0,0,0,0,3,0,0
747969,2.020110e+13,44997,04.22.1750,0.0,0.0,0.0,0,0,0.0,0,0,0,0,3,0,0
747970,2.020110e+13,44997,04.22.1750,0.0,0.0,0.0,0,0,0.0,0,0,0,0,3,0,0


In [29]:
test_qul.isnull().sum()

time               0
user_id            0
fwver          22764
quality_0     106584
quality_1         11
quality_2      21115
quality_3          0
quality_4          0
quality_5         44
quality_6          0
quality_7          0
quality_8          0
quality_9          0
quality_10         0
quality_11         0
quality_12         0
dtype: int64

In [30]:
test_qul.quality_0 = test_qul.quality_0.fillna(0)
test_qul.quality_2 = test_qul.quality_2.fillna(0)
test_qul.quality_5 = test_qul.quality_5.fillna(0)

In [31]:
test_qul.isnull().sum()

time              0
user_id           0
fwver         22764
quality_0         0
quality_1        11
quality_2         0
quality_3         0
quality_4         0
quality_5         0
quality_6         0
quality_7         0
quality_8         0
quality_9         0
quality_10        0
quality_11        0
quality_12        0
dtype: int64

In [32]:
test_qul = test_qul.dropna()

In [33]:
test_qul.isnull().sum()

time          0
user_id       0
fwver         0
quality_0     0
quality_1     0
quality_2     0
quality_3     0
quality_4     0
quality_5     0
quality_6     0
quality_7     0
quality_8     0
quality_9     0
quality_10    0
quality_11    0
quality_12    0
dtype: int64

In [34]:
test_qul.fwver.unique()

array(['04.33.1261', '05.15.2138', '04.22.1750', '04.22.1778',
       '04.16.3553', '09.17.1431', '03.11.1167', '04.33.1149',
       '04.33.1185', '04.22.1684', '04.82.1684', '04.16.3571',
       '04.73.2571', '04.82.1778', '04.73.2237', '05.66.3237',
       '05.66.3571', '03.11.1149', '05.15.2120', '04.33.1125',
       '04.16.3439', '05.15.2114'], dtype=object)

In [35]:
test_qul

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,2.020110e+13,30000,04.33.1261,0.0,0.0,0.0,0,0,0.0,0,5,0,0,2,0,0
1,2.020110e+13,30000,04.33.1261,0.0,0.0,0.0,0,0,0.0,0,5,0,0,2,0,0
2,2.020110e+13,30000,04.33.1261,0.0,0.0,0.0,0,0,0.0,0,5,0,0,2,0,0
3,2.020110e+13,30000,04.33.1261,0.0,0.0,0.0,0,0,0.0,0,5,0,0,2,0,0
4,2.020110e+13,30000,04.33.1261,0.0,0.0,0.0,0,0,0.0,0,5,0,0,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747967,2.020110e+13,44997,04.22.1750,0.0,0.0,0.0,0,0,2.0,0,0,0,0,3,0,0
747968,2.020110e+13,44997,04.22.1750,0.0,0.0,0.0,0,0,0.0,0,0,0,0,3,0,0
747969,2.020110e+13,44997,04.22.1750,0.0,0.0,0.0,0,0,0.0,0,0,0,0,3,0,0
747970,2.020110e+13,44997,04.22.1750,0.0,0.0,0.0,0,0,0.0,0,0,0,0,3,0,0


In [36]:
for x in range(16): 
    print(test_qul.iloc[:,x].value_counts())

2.020110e+13    725065
2.020100e+13       132
Name: time, dtype: int64
37632    3792
32690    3408
33783    3264
36582    3204
41954    2988
         ... 
41110      12
34558      12
39164      12
35070      12
32784      12
Name: user_id, Length: 8219, dtype: int64
05.15.2138    142693
04.22.1750    141912
04.33.1261    121584
04.16.3553    109056
03.11.1167     84240
04.22.1778     59736
04.33.1185     57828
04.16.3571      2172
04.22.1684      1464
03.11.1149      1188
09.17.1431      1116
04.82.1684       900
04.33.1149       444
04.82.1778       372
05.66.3237       144
04.73.2237       132
05.15.2120       108
05.66.3571        36
04.73.2571        36
05.15.2114        12
04.16.3439        12
04.33.1125        12
Name: fwver, dtype: int64
 0.0        589935
-1.0        126602
 1.0          2316
 2.0          1564
 3.0           739
             ...  
 1189.0          1
 19059.0         1
 1202.0          1
 301.0           1
 166.0           1
Name: quality_0, Length: 529, dtype:

In [37]:
test_qul_0 = test_qul.replace(-1,0)

In [38]:
test_quality_user_id_fwver_groupby = test_qul_0.groupby(['user_id', 'fwver'])[
    "quality_0","quality_1","quality_2",
    "quality_5","quality_6","quality_7","quality_8",
    "quality_9","quality_10","quality_11","quality_12"].mean()

In [39]:
test_quality_fwver_groupby = test_qul_0.groupby(['fwver'])[
    "quality_0","quality_1","quality_2",
    "quality_5","quality_6","quality_7","quality_8",
    "quality_9","quality_10","quality_11","quality_12"].mean()

In [40]:
test_quality_user_id_fwver_groupby

quality_0  quality_1  quality_2  quality_5  quality_6  \
user_id fwver                                                               
30000   04.33.1261     0.0000        0.0     0.0000   0.166667   0.416667   
30001   05.15.2138     0.0000        0.0     0.0000  71.194444   0.000000   
30002   04.22.1750     0.0000        0.0     0.0000   1.108333   0.000000   
30003   04.22.1750     0.0000        0.0     0.0000   0.166667   4.444444   
        04.22.1778     0.0000        0.0     0.0000   0.250000   2.500000   
...                       ...        ...        ...        ...        ...   
44993   05.15.2138     0.1875        0.0     0.1875   0.291667   2.270833   
44994   04.16.3553     0.0000        0.0     0.0000   0.166667   0.000000   
44996   04.22.1750     0.0000        0.0     0.0000   0.300000   0.000000   
        04.22.1778     0.0000        0.0     0.0000   0.395833   0.000000   
44997   04.22.1750     0.0000        0.0     0.0000   0.541667   0.000000   

                    quality_7  quality_8  quality_9  quality_10  quality_11  \
user_id fwver                                                                 
30000   04.33.1261   5.000000        0.0       0.00    2.000000         0.0   
30001   05.15.2138   0.000000        0.0       0.00  854.333333         0.0   
30002   04.22.1750   0.000000        0.0       0.00   13.300000         0.0   
30003   04.22.1750  53.333333        0.0       0.00    2.000000         0.0   
        04.22.1778  30.000000        0.0       0.00    3.000000         0.0   
...                       ...        ...        ...         ...         ...   
44993   05.15.2138  27.250000        0.0       2.25    3.500000         0.0   
44994   04.16.3553   0.000000        0.0       0.00    2.000000         0.0   
44996   04.22.1750   0.000000        0.0       0.00    3.600000         0.0   
        04.22.1778   0.000000        0.0       0.00    4.750000         0.0   
44997   04.22.1750   0.000000        0.0       0.00    6.500000         0.0   

                    quality_12  
user_id fwver                   
30000   04.33.1261         0.0  
30001   05.15.2138         0.0  
30002   04.22.1750         0.0  
30003   04.22.1750         0.0  
        04.22.1778         0.0  
...                        ...  
44993   05.15.2138         0.0  
44994   04.16.3553         0.0  
44996   04.22.1750         0.0  
        04.22.1778         0.0  
44997   04.22.1750         0.0  

[10059 rows x 11 columns]

In [41]:
test_quality_fwver_groupby

,quality_0,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
fwver,,,,,,,,,,,
03.11.1149,0.000000,0.000000,0.000000,0.176768,0.000000,0.000000,0.000000,0.000000,2.121212,0.000000,0.000000
03.11.1167,0.000000,0.063094,40.458179,8.975427,0.365907,4.390883,0.757123,485.498148,107.705128,0.020821,0.249858
04.16.3439,0.000000,0.000000,0.000000,0.916667,0.000000,0.000000,0.000000,0.000000,11.000000,0.000000,0.000000
04.16.3553,0.000000,0.016872,0.170765,37.516835,1.658203,19.898438,0.202465,2.049186,450.202025,0.000303,0.003631
04.16.3571,0.000000,0.033610,0.000000,23.941068,13.072284,156.867403,0.403315,0.000000,287.292818,0.000000,0.000000
04.22.1684,0.000000,0.000000,0.000000,21.000000,12.056694,144.680328,0.000000,0.000000,252.000000,0.000000,0.000000
04.22.1750,0.000000,0.011084,0.000000,144.579204,2.563286,30.759428,0.133012,0.000000,1734.950448,0.000155,0.001860
04.22.1778,0.000000,0.016205,0.000000,38.612913,1.077993,12.935918,0.194456,0.000000,463.354962,0.001189,0.014263
04.33.1125,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000


## test id fwver별로 묶을 인덱스 불러오기

In [42]:
test_err  = pd.read_csv('LG/err_group_test.csv')
test_err

,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,...,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,time_min,time_max,user_id,model_nm,fwver
0,0.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,46.0,0.0,0.0,20201101030227,20201127024946,30000,model_1,04.16.3553
1,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,42.0,0.0,1.0,20201118073855,20201122050431,30000,model_1,04.16.3571
2,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,1.0,20201127094708,20201130192105,30000,model_2,04.33.1261
3,0.0,0.0,3.0,98.0,3.0,4.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,20201101042135,20201130221906,30001,model_3,05.15.2138
4,0.0,0.0,0.0,0.0,31.0,2.0,1.0,0.0,0.0,0.0,...,0.0,14.0,95.0,1.0,0.0,20201101003616,20201126040300,30002,model_0,04.22.1750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24091,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,156.0,1.0,0.0,20201101003242,20201119052538,44996,model_0,04.22.1750
24092,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,173.0,1.0,3.0,20201119053129,20201130220212,44996,model_0,04.22.1778
24093,0.0,0.0,0.0,1.0,1.0,1.0,2.0,0.0,0.0,0.0,...,3.0,0.0,28.0,44.0,0.0,20201101000032,20201127035735,44997,model_0,04.22.1750
24094,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,5.0,0.0,20201127040356,20201130232034,44997,model_0,04.22.1778


In [43]:
test_err_name = test_err.groupby(["user_id","fwver"]).sum()
test_err_name = pd.DataFrame(test_err_name.index)

In [44]:
test_err_name

,0
0,"(30000, 04.16.3553)"
1,"(30000, 04.16.3571)"
2,"(30000, 04.33.1261)"
3,"(30001, 05.15.2138)"
4,"(30002, 04.22.1750)"
...,...
24091,"(44996, 04.22.1750)"
24092,"(44996, 04.22.1778)"
24093,"(44997, 04.22.1750)"
24094,"(44997, 04.22.1778)"


In [45]:
a = pd.DataFrame(np.empty((1 ,2), object))
for x in range(len(test_err_name)):
    b = pd.DataFrame(test_err_name.iloc[x,0]).T
    a = a.append(b)

In [46]:
test_err_name = a.dropna()

In [47]:
test_err_name.rename(columns = {0 : 'user_id'}, inplace = True)
test_err_name.rename(columns = {1 : 'fwver'}, inplace = True)

In [48]:
test_err_name

,user_id,fwver
0,30000,04.16.3553
0,30000,04.16.3571
0,30000,04.33.1261
0,30001,05.15.2138
0,30002,04.22.1750
...,...,...
0,44996,04.22.1750
0,44996,04.22.1778
0,44997,04.22.1750
0,44997,04.22.1778


## Test quality data -1 제외 평균낸거 인덱싱

In [49]:
test_quality_user_id_fwver_merge = pd.merge(test_err_name,test_quality_user_id_fwver_groupby, on=['user_id','fwver'], how ='left')

In [50]:
test_quality_fwver_merge = pd.merge(test_err_name,test_quality_fwver_groupby, on=['fwver'], how ='left')

In [51]:
test_quality_user_id_fwver_merge.isnull().sum()

user_id           0
fwver             0
quality_0     14076
quality_1     14076
quality_2     14076
quality_5     14076
quality_6     14076
quality_7     14076
quality_8     14076
quality_9     14076
quality_10    14076
quality_11    14076
quality_12    14076
dtype: int64

In [52]:
test_quality_fwver_merge.isnull().sum()

user_id        0
fwver          0
quality_0     81
quality_1     81
quality_2     81
quality_5     81
quality_6     81
quality_7     81
quality_8     81
quality_9     81
quality_10    81
quality_11    81
quality_12    81
dtype: int64

In [53]:
test_quality_user_id_fwver_merge['quality_0']=test_quality_user_id_fwver_merge['quality_0'].fillna(test_quality_fwver_merge['quality_0'])
test_quality_user_id_fwver_merge['quality_1']=test_quality_user_id_fwver_merge['quality_1'].fillna(test_quality_fwver_merge['quality_1'])
test_quality_user_id_fwver_merge['quality_2']=test_quality_user_id_fwver_merge['quality_2'].fillna(test_quality_fwver_merge['quality_2'])
test_quality_user_id_fwver_merge['quality_5']=test_quality_user_id_fwver_merge['quality_5'].fillna(test_quality_fwver_merge['quality_5'])
test_quality_user_id_fwver_merge['quality_6']=test_quality_user_id_fwver_merge['quality_6'].fillna(test_quality_fwver_merge['quality_6'])
test_quality_user_id_fwver_merge['quality_7']=test_quality_user_id_fwver_merge['quality_7'].fillna(test_quality_fwver_merge['quality_7'])
test_quality_user_id_fwver_merge['quality_8']=test_quality_user_id_fwver_merge['quality_8'].fillna(test_quality_fwver_merge['quality_8'])
test_quality_user_id_fwver_merge['quality_9']=test_quality_user_id_fwver_merge['quality_9'].fillna(test_quality_fwver_merge['quality_9'])
test_quality_user_id_fwver_merge['quality_10']=test_quality_user_id_fwver_merge['quality_10'].fillna(test_quality_fwver_merge['quality_10'])
test_quality_user_id_fwver_merge['quality_11']=test_quality_user_id_fwver_merge['quality_11'].fillna(test_quality_fwver_merge['quality_11'])
test_quality_user_id_fwver_merge['quality_12']=test_quality_user_id_fwver_merge['quality_12'].fillna(test_quality_fwver_merge['quality_12'])
test_quality_user_id_fwver_merge = test_quality_user_id_fwver_merge.fillna(0)

In [54]:
test_quality_user_id_fwver_merge

,user_id,fwver,quality_0,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,30000,04.16.3553,0.0,0.016872,0.170765,37.516835,1.658203,19.898438,0.202465,2.049186,450.202025,0.000303,0.003631
1,30000,04.16.3571,0.0,0.033610,0.000000,23.941068,13.072284,156.867403,0.403315,0.000000,287.292818,0.000000,0.000000
2,30000,04.33.1261,0.0,0.000000,0.000000,0.166667,0.416667,5.000000,0.000000,0.000000,2.000000,0.000000,0.000000
3,30001,05.15.2138,0.0,0.000000,0.000000,71.194444,0.000000,0.000000,0.000000,0.000000,854.333333,0.000000,0.000000
4,30002,04.22.1750,0.0,0.000000,0.000000,1.108333,0.000000,0.000000,0.000000,0.000000,13.300000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24091,44996,04.22.1750,0.0,0.000000,0.000000,0.300000,0.000000,0.000000,0.000000,0.000000,3.600000,0.000000,0.000000
24092,44996,04.22.1778,0.0,0.000000,0.000000,0.395833,0.000000,0.000000,0.000000,0.000000,4.750000,0.000000,0.000000
24093,44997,04.22.1750,0.0,0.000000,0.000000,0.541667,0.000000,0.000000,0.000000,0.000000,6.500000,0.000000,0.000000
24094,44997,04.22.1778,0.0,0.016205,0.000000,38.612913,1.077993,12.935918,0.194456,0.000000,463.354962,0.001189,0.014263


In [55]:
test_quality_user_id_fwver_merge.to_csv('test_quality_merge.csv',index=False)

----
## Test quality data -1 갯수 세기

In [56]:
test_qul  = pd.read_csv('LG/test_quality_data.csv')
test_qul

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,2.020110e+13,30000,04.33.1261,0.0,0.0,0.0,0,0,0.0,0,5,0,0,2,0,0
1,2.020110e+13,30000,04.33.1261,0.0,0.0,0.0,0,0,0.0,0,5,0,0,2,0,0
2,2.020110e+13,30000,04.33.1261,0.0,0.0,0.0,0,0,0.0,0,5,0,0,2,0,0
3,2.020110e+13,30000,04.33.1261,0.0,0.0,0.0,0,0,0.0,0,5,0,0,2,0,0
4,2.020110e+13,30000,04.33.1261,0.0,0.0,0.0,0,0,0.0,0,5,0,0,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747967,2.020110e+13,44997,04.22.1750,0.0,0.0,0.0,0,0,2.0,0,0,0,0,3,0,0
747968,2.020110e+13,44997,04.22.1750,0.0,0.0,0.0,0,0,0.0,0,0,0,0,3,0,0
747969,2.020110e+13,44997,04.22.1750,0.0,0.0,0.0,0,0,0.0,0,0,0,0,3,0,0
747970,2.020110e+13,44997,04.22.1750,0.0,0.0,0.0,0,0,0.0,0,0,0,0,3,0,0


In [57]:
for i in range(13):
    original_column = 'quality_' + str(i)
    new_column = 'quality_' + str(i) + str('_minus')
    test_qul[new_column]=0
    for j in tqdm(range(747972)):
        if test_qul[original_column][j] == -1:
            test_qul[new_column][j] = 1
            test_qul[original_column][j] = 0

100%|███████████████████████████████████████████████████████████████████████| 747972/747972 [00:18<00:00, 40898.69it/s]


In [58]:
test_qul

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,...,quality_3_minus,quality_4_minus,quality_5_minus,quality_6_minus,quality_7_minus,quality_8_minus,quality_9_minus,quality_10_minus,quality_11_minus,quality_12_minus
0,2.020110e+13,30000,04.33.1261,0.0,0.0,0.0,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
1,2.020110e+13,30000,04.33.1261,0.0,0.0,0.0,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
2,2.020110e+13,30000,04.33.1261,0.0,0.0,0.0,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
3,2.020110e+13,30000,04.33.1261,0.0,0.0,0.0,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
4,2.020110e+13,30000,04.33.1261,0.0,0.0,0.0,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747967,2.020110e+13,44997,04.22.1750,0.0,0.0,0.0,0,0,2.0,0,...,0,0,0,0,0,0,0,0,0,0
747968,2.020110e+13,44997,04.22.1750,0.0,0.0,0.0,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
747969,2.020110e+13,44997,04.22.1750,0.0,0.0,0.0,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
747970,2.020110e+13,44997,04.22.1750,0.0,0.0,0.0,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0


In [59]:
test_qul0 = test_qul.drop(["quality_0","quality_1","quality_2","quality_3","quality_4",
    "quality_5","quality_6","quality_7","quality_8",
    "quality_9","quality_10","quality_11","quality_12","quality_3_minus", "quality_4_minus"],axis=1)

In [60]:
test_qul0

,time,user_id,fwver,quality_0_minus,quality_1_minus,quality_2_minus,quality_5_minus,quality_6_minus,quality_7_minus,quality_8_minus,quality_9_minus,quality_10_minus,quality_11_minus,quality_12_minus
0,2.020110e+13,30000,04.33.1261,0,0,0,0,0,0,0,0,0,0,0
1,2.020110e+13,30000,04.33.1261,0,0,0,0,0,0,0,0,0,0,0
2,2.020110e+13,30000,04.33.1261,0,0,0,0,0,0,0,0,0,0,0
3,2.020110e+13,30000,04.33.1261,0,0,0,0,0,0,0,0,0,0,0
4,2.020110e+13,30000,04.33.1261,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747967,2.020110e+13,44997,04.22.1750,0,0,0,0,0,0,0,0,0,0,0
747968,2.020110e+13,44997,04.22.1750,0,0,0,0,0,0,0,0,0,0,0
747969,2.020110e+13,44997,04.22.1750,0,0,0,0,0,0,0,0,0,0,0
747970,2.020110e+13,44997,04.22.1750,0,0,0,0,0,0,0,0,0,0,0


In [61]:
for x in range(14): 
    print(test_qul0.iloc[:,x].value_counts())

2.020110e+13    747840
2.020100e+13       132
Name: time, dtype: int64
37632    3792
32690    3408
33783    3264
36582    3204
41954    2988
         ... 
41246      12
32518      12
36120      12
32006      12
32784      12
Name: user_id, Length: 8268, dtype: int64
05.15.2138    142704
04.22.1750    141912
04.33.1261    121584
04.16.3553    109056
03.11.1167     84240
04.22.1778     59736
04.33.1185     57828
04.16.3571      2172
04.22.1684      1464
03.11.1149      1188
09.17.1431      1116
04.82.1684       900
04.33.1149       444
04.82.1778       372
05.66.3237       144
04.73.2237       132
05.15.2120       108
05.66.3571        36
04.73.2571        36
05.15.2114        12
04.16.3439        12
04.33.1125        12
Name: fwver, dtype: int64
0    620971
1    127001
Name: quality_0_minus, dtype: int64
0    606067
1    141905
Name: quality_1_minus, dtype: int64
0    611108
1    136864
Name: quality_2_minus, dtype: int64
0    606184
1    141788
Name: quality_5_minus, dtype: int64
0    

In [62]:
test_quality_user_id_fwver_groupby_minus = test_qul0.groupby(['user_id', 'fwver'])[
    "quality_0_minus","quality_1_minus","quality_2_minus",
    "quality_6_minus","quality_11_minus"].mean()

In [63]:
test_quality_fwver_groupby_minus = test_qul0.groupby(['fwver'])[
    "quality_0_minus","quality_1_minus","quality_2_minus",
    "quality_6_minus","quality_11_minus"].mean()

In [64]:
test_quality_user_id_fwver_groupby_minus

quality_0_minus  quality_1_minus  quality_2_minus  \
user_id fwver                                                           
30000   04.33.1261         0.000000         0.000000         0.000000   
30001   05.15.2138         0.250000         0.250000         0.250000   
30002   04.22.1750         0.125000         0.125000         0.125000   
30003   04.22.1750         0.000000         0.000000         0.000000   
        04.22.1778         0.000000         0.000000         0.000000   
...                             ...              ...              ...   
44993   05.15.2138         0.041667         0.041667         0.041667   
44994   04.16.3553         0.916667         0.916667         0.916667   
44996   04.22.1750         0.000000         0.000000         0.000000   
        04.22.1778         0.104167         0.104167         0.104167   
44997   04.22.1750         0.000000         0.000000         0.000000   

                    quality_6_minus  quality_11_minus  
user_id fwver                                          
30000   04.33.1261         0.000000          0.000000  
30001   05.15.2138         0.250000          0.250000  
30002   04.22.1750         0.125000          0.125000  
30003   04.22.1750         0.000000          0.000000  
        04.22.1778         0.000000          0.000000  
...                             ...               ...  
44993   05.15.2138         0.041667          0.041667  
44994   04.16.3553         0.916667          0.916667  
44996   04.22.1750         0.000000          0.000000  
        04.22.1778         0.104167          0.104167  
44997   04.22.1750         0.000000          0.000000  

[10059 rows x 5 columns]

In [65]:
test_quality_fwver_groupby_minus

,quality_0_minus,quality_1_minus,quality_2_minus,quality_6_minus,quality_11_minus
fwver,,,,,
03.11.1149,0.000000,0.000000,0.000000,0.000000,0.000000
03.11.1167,0.000000,0.117082,0.117082,0.117082,0.117082
04.16.3439,0.000000,0.000000,0.000000,0.000000,0.000000
04.16.3553,0.178413,0.178413,0.178413,0.178413,0.178413
04.16.3571,0.162063,0.162063,0.162063,0.162063,0.162063
04.22.1684,0.013661,0.013661,0.013661,0.013661,0.013661
04.22.1750,0.215704,0.215577,0.215704,0.215591,0.215591
04.22.1778,0.224203,0.224036,0.224203,0.224002,0.224002
04.33.1125,0.500000,0.500000,0.500000,0.500000,0.500000


In [66]:
test_id_fwver_merge_minus = pd.merge(test_err_name,test_quality_user_id_fwver_groupby_minus, on=['user_id','fwver'], how ='left')

In [67]:
test_fwver_merge_minus = pd.merge(test_err_name,test_quality_fwver_groupby_minus, on=['fwver'], how ='left')

In [68]:
test_id_fwver_merge_minus

,user_id,fwver,quality_0_minus,quality_1_minus,quality_2_minus,quality_6_minus,quality_11_minus
0,30000,04.16.3553,NaN,NaN,NaN,NaN,NaN
1,30000,04.16.3571,NaN,NaN,NaN,NaN,NaN
2,30000,04.33.1261,0.000000,0.000000,0.000000,0.000000,0.000000
3,30001,05.15.2138,0.250000,0.250000,0.250000,0.250000,0.250000
4,30002,04.22.1750,0.125000,0.125000,0.125000,0.125000,0.125000
...,...,...,...,...,...,...,...
24091,44996,04.22.1750,0.000000,0.000000,0.000000,0.000000,0.000000
24092,44996,04.22.1778,0.104167,0.104167,0.104167,0.104167,0.104167
24093,44997,04.22.1750,0.000000,0.000000,0.000000,0.000000,0.000000
24094,44997,04.22.1778,NaN,NaN,NaN,NaN,NaN


In [ ]:
test_fwver_merge_minus

In [69]:
test_id_fwver_merge_minus['quality_0_minus']=test_id_fwver_merge_minus['quality_0_minus'].fillna(test_fwver_merge_minus['quality_0_minus'])
test_id_fwver_merge_minus['quality_1_minus']=test_id_fwver_merge_minus['quality_1_minus'].fillna(test_fwver_merge_minus['quality_1_minus'])
test_id_fwver_merge_minus['quality_2_minus']=test_id_fwver_merge_minus['quality_2_minus'].fillna(test_fwver_merge_minus['quality_2_minus'])
test_id_fwver_merge_minus['quality_6_minus']=test_id_fwver_merge_minus['quality_6_minus'].fillna(test_fwver_merge_minus['quality_6_minus'])
test_id_fwver_merge_minus['quality_11_minus']=test_id_fwver_merge_minus['quality_11_minus'].fillna(test_fwver_merge_minus['quality_11_minus'])

In [70]:
test_id_fwver_merge_minus = test_id_fwver_merge_minus.fillna(0)

In [73]:
test_id_fwver_merge_minus.to_csv('test_quality_merge_minus.csv',index=False)

In [ ]:
test_id_fwver_merge_minus